# Data Analysis Case Study - Apply Problem Solving Strategy

**Problem**
Assume you are in the role of an Analyst in the product development of the company. To help increase the volume of orders and customers purchasing movie tickets online, you need to analyze data of customers' booking history over the past years. From there, provide informative insights about customer behavior and corresponding recommendations.

This is a capstone project I completed for self-learning Python for Data Analysis, utilizing industry-standard libraries: Pandas and Matplotlib.
There are many frameworks for data analysis. Framework provides a structured and systemetic approach for managing and analyzing data tasks/projects to ensure consistency, efficiency, and high quality work. The framework I used is: 

Problem Definition => Data Collection => Data Transformation => Data Exploration => Sharing Insights

## I. Define The Problem 
### Why?
- The objective of this task is to analyze user behavior in purchasing movie tickets over the past four years using data.
- By gaining this understanding, we can develop insights to implement effective strategies to increase sales and the number of buyers in the coming years.

### Who?
- Internal: Departments and/or staffs involves in ticket purchasing process? Sales, customer service
- External: Who is our customers?
-  Location: which city/province/area
-  Profile: new customer, old customer, activated customer,..
-  Demographic: gender (male/femail), education, marriage or not,..

### What?
- Analyze ticket purchasing behavior using website and mobile app

### Which
Which element or factor  does customers use duing online ticket purchasing process?
-- Product: tickets
-- Device: phone, website
-- Payment options: cash, card, bank transfer, e-wallet 
-- Price
-- Promotion / Discount

### When
When do customers often buy tickets?
- Year, month, day, hour
- Holiday, event, new movie

### How
How did customers use the product?
- Customer experience: Good, bad
++ Feedback
++ %Successful ticket purchases
++ %Customer retention
- Purchasing process
++ Total time completing ticket purchase
++ Time taken for each step in the process. Any bottle-necks? Any step that customer faced frustration?

# II. Disaggregate The Problem
Based on the problem definition, we can create a Logic Tree

## 1. Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [42]:
df_customer = pd.read_csv("D:/Git DucD Data/SQLplaygroundDataAnalysis/Capstone project/Dataset/customer.csv")
df_campaign = pd.read_csv("D:/Git DucD Data/SQLplaygroundDataAnalysis/Capstone project/Dataset/campaign.csv")
df_device = pd.read_csv("D:/Git DucD Data/SQLplaygroundDataAnalysis/Capstone project/Dataset/device_detail.csv")
df_sales = pd.read_csv("D:/Git DucD Data/SQLplaygroundDataAnalysis/Capstone project/Dataset/sales.csv")
df_status = pd.read_csv("D:/Git DucD Data/SQLplaygroundDataAnalysis/Capstone project/Dataset/status_detail.csv")
df_ticket = pd.read_csv("D:/Git DucD Data/SQLplaygroundDataAnalysis/Capstone project/Dataset/ticket_history.csv")

## 2. Clean Data

**2.1 Data type, NULL value, duplicate values**

In [13]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52054 entries, 0 to 52053
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     52054 non-null  int64 
 1   usergender  52054 non-null  int64 
 2   dob         52054 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.2+ MB


In [43]:
#CUSTOMER table
#convert DOB type to datetime from object type
import datetime
df_customer['dob'] = pd.to_datetime(df_customer['dob'])

In [17]:
#to check duplicate, we unique count records and compare with the total record number
df_customer.nunique()

user_id       52054
usergender        3
dob            9115
dtype: int64

In [45]:
#CAMPAIGN table
df_campaign.info()
df_campaign.nunique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   campaign_code  105 non-null    int64 
 1   campaign_type  105 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


campaign_code    105
campaign_type      3
dtype: int64

In [53]:
#DEVICE table
df_device.info()

#from the result, we can see some null numbers in model 55519 comparing to other columns
#before making decision on how to handle we need to check %null value on total

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55639 entries, 0 to 55638
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   device_id  55637 non-null  object
 1   model      55519 non-null  object
 2   platform   55639 non-null  object
dtypes: object(3)
memory usage: 1.3+ MB


In [63]:
def calc_null_rate(df):
    """check NULL rate of each column
    """
    newdf = df.isnull().sum().to_frame('null count')
    newdf [['null rate']] = newdf[['null count']] / len(df)
    return newdf.sort_values(by=['null rate'], ascending=False)



In [ ]:
#calculate null and null rate
calc_null_rate(df_device)

,null count,null rate
model,120,0.002157
device_id,2,0.000036
platform,0,0.000000


In [79]:
#[Duc note] for data analysis, delete 2 records that have null value has minor impact of our analysis.

#change NULL value in model to unknown string
df_device = df_device.fillna({'model' : 'unknown'})

#delete 2 NULL records in devicemodel
df_device = df_device[df_device['device_id'].notna()]

calc_null_rate(df_device)

,null count,null rate
device_id,0,0.0
model,0,0.0
platform,0,0.0


In [88]:
#STATUS table
df_status
df_status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   status_id    8 non-null      int64 
 1   status_name  8 non-null      object
 2   description  8 non-null      object
 3   error_group  7 non-null      object
dtypes: int64(1), object(3)
memory usage: 388.0+ bytes


In [ ]:
#TICKET HISTORY table (fact)
#[Duc] as it's a fact table - transaction records, we need to review more throughly
df_ticket
df_ticket.info()
df_ticket.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67522 entries, 0 to 67521
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ticket_id          67522 non-null  object 
 1   customer_id        67522 non-null  int64  
 2   paying_method      67522 non-null  object 
 3   theater_name       67522 non-null  float64
 4   device_id          67522 non-null  object 
 5   original_price     67522 non-null  float64
 6   discount_value     67522 non-null  float64
 7   final_price        67522 non-null  float64
 8   time               67522 non-null  object 
 9   status_id          67522 non-null  int64  
 10  mkt_campaign_code  67522 non-null  int64  
 11  movie_name         67522 non-null  object 
dtypes: float64(4), int64(3), object(5)
memory usage: 6.2+ MB


In [96]:
#Time table is in object/text. It should be time
df_ticket['time'] = pd.to_datetime(df_ticket['time'])

In [97]:
df_ticket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67522 entries, 0 to 67521
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ticket_id          67522 non-null  object        
 1   customer_id        67522 non-null  int64         
 2   paying_method      67522 non-null  object        
 3   theater_name       67522 non-null  float64       
 4   device_id          67522 non-null  object        
 5   original_price     67522 non-null  float64       
 6   discount_value     67522 non-null  float64       
 7   final_price        67522 non-null  float64       
 8   time               67522 non-null  datetime64[ns]
 9   status_id          67522 non-null  int64         
 10  mkt_campaign_code  67522 non-null  int64         
 11  movie_name         67522 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(4)
memory usage: 6.2+ MB


In [99]:
calc_null_rate(df_ticket)

,null count,null rate
ticket_id,0,0.0
customer_id,0,0.0
paying_method,0,0.0
theater_name,0,0.0
device_id,0,0.0
original_price,0,0.0
discount_value,0,0.0
final_price,0,0.0
time,0,0.0
status_id,0,0.0


In [103]:
df_ticket.nunique()
#total records: 67522 ; there are duplicate records

ticket_id            67515
customer_id          50512
paying_method            4
theater_name           114
device_id            53352
original_price        1003
discount_value         176
final_price           1438
time                 67515
status_id                8
mkt_campaign_code      103
movie_name             316
dtype: int64

In [104]:
#check duplicate
df_dup = df_ticket[df_ticket.duplicated(keep=False)]

df_dup

,ticket_id,customer_id,paying_method,theater_name,device_id,original_price,discount_value,final_price,time,status_id,mkt_campaign_code,movie_name
24027,de40b1921df46e455e426a6dac047d4d,105710,balance,95.0,2df2d3e16931fe66f8626a7778dfde9b,4.54,0.0,4.54,2019-12-23 10:47:42.317,1,0,Dreamy Eyes
24028,de40b1921df46e455e426a6dac047d4d,105710,balance,95.0,2df2d3e16931fe66f8626a7778dfde9b,4.54,0.0,4.54,2019-12-23 10:47:42.317,1,0,Dreamy Eyes
24111,763e20c9c1136c5c06d9a960fac9dae6,143762,credit card,17.0,ba5794fca2cba47fd0141dd3fb195934,7.42,0.0,7.42,2019-12-23 15:28:45.738,1,0,Dreamy Eyes
24112,763e20c9c1136c5c06d9a960fac9dae6,143762,credit card,17.0,ba5794fca2cba47fd0141dd3fb195934,7.42,0.0,7.42,2019-12-23 15:28:45.738,1,0,Dreamy Eyes
24139,a6062e2443611f0d65727c5469fe89a2,109678,bank account,53.0,7f38dbd65e6c83116f7dd7a7cbe5bf16,9.07,0.0,9.07,2019-12-23 17:15:52.991,1,0,Dreamy Eyes
24140,a6062e2443611f0d65727c5469fe89a2,109678,bank account,53.0,7f38dbd65e6c83116f7dd7a7cbe5bf16,9.07,0.0,9.07,2019-12-23 17:15:52.991,1,0,Dreamy Eyes
24189,d0559f718b50247ad965767ea9be3ab3,131072,balance,112.0,0b667fb51d16c8a931b92e17084304f6,5.36,0.0,5.36,2019-12-23 23:15:06.060,1,0,Dreamy Eyes
24190,d0559f718b50247ad965767ea9be3ab3,131072,balance,112.0,0b667fb51d16c8a931b92e17084304f6,5.36,0.0,5.36,2019-12-23 23:15:06.060,1,0,Dreamy Eyes
24422,2cdf7de498105664abbd91bc30bb4438,100194,bank account,113.0,769e0ddeac9b9d84ac49c2588b537900,7.42,0.0,7.42,2019-12-23 17:33:03.167,1,0,Dreamy Eyes
24423,2cdf7de498105664abbd91bc30bb4438,100194,bank account,113.0,769e0ddeac9b9d84ac49c2588b537900,7.42,0.0,7.42,2019-12-23 17:33:03.167,1,0,Dreamy Eyes


In [ ]:
#We have records that have the same information ??
df_ticket.drop_duplicates(inplace=True) #remove duplicate. default keep the first one. inplace true to modify the original data frame


In [ ]:
df_ticket.nunique()
df_ticket.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67515 entries, 0 to 67521
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ticket_id          67515 non-null  object        
 1   customer_id        67515 non-null  int64         
 2   paying_method      67515 non-null  object        
 3   theater_name       67515 non-null  float64       
 4   device_id          67515 non-null  object        
 5   original_price     67515 non-null  float64       
 6   discount_value     67515 non-null  float64       
 7   final_price        67515 non-null  float64       
 8   time               67515 non-null  datetime64[ns]
 9   status_id          67515 non-null  int64         
 10  mkt_campaign_code  67515 non-null  int64         
 11  movie_name         67515 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(4)
memory usage: 6.7+ MB


In [162]:
#[Duc]join all table to give us a systematic view of all our data instead of silo tables
print(df_join_campaign.columns)
print(df_device.columns)

#from tickets we join with the other tables
df_join_customer = pd.merge(df_ticket, df_customer, how = 'left',  left_on='customer_id', right_on='user_id')
df_join_campaign = pd.merge(df_join_customer, df_campaign, how = 'left', left_on='mkt_campaign_code', right_on='campaign_code')
df_join_status = pd.merge(df_join_campaign, df_status, how = 'left', on='status_id')
df_join_all = pd.merge(df_join_status, df_device, how = 'left', on='device_id')

Index(['ticket_id', 'customer_id', 'paying_method', 'theater_name',
       'device_id', 'original_price', 'discount_value', 'final_price', 'time',
       'status_id', 'mkt_campaign_code', 'movie_name', 'user_id', 'usergender',
       'dob', 'campaign_code', 'campaign_type'],
      dtype='object')
Index(['device_id', 'model', 'platform'], dtype='object')


In [165]:
df_join_all.head(2)

,ticket_id,customer_id,paying_method,theater_name,device_id,original_price,discount_value,final_price,time,status_id,...,user_id,usergender,dob,campaign_code,campaign_type,status_name,description,error_group,model,platform
0,9f2da3491b42b7599065ac102bdcb4c2,100004,credit card,33.0,3cac5d2e2eb76525aecea5c2ab46b3d9,4.54,1.28,3.26,2019-11-16 13:28:24.970,1,...,100004,1,1989-02-25,25690.0,direct discount,SUCCESS,Transaction successful,NaN,"iPhone10,2",mobile
1,8607fe9d2d12e14d711c49ae1204de29,118325,credit card,20.0,9d6100d117d5f0aa65fc29b6d9edd9f7,3.09,0.00,3.09,2019-11-13 18:12:03.260,1,...,118325,2,1992-10-03,NaN,NaN,SUCCESS,Transaction successful,NaN,"iPhone10,6",mobile


In [ ]:
calc_null_rate(df_join_all)

,null count,null rate
error_group,57746,0.834528
campaign_code,33711,0.487181
campaign_type,33711,0.487181
model,178,0.002572
platform,178,0.002572
ticket_id,0,0.000000
original_price,0,0.000000
device_id,0,0.000000
theater_name,0,0.000000
paying_method,0,0.000000


**Analysis:**
Looking at the attributes that have NULL, 
- error_group: We can ignore 'error_group' because it identifies if a records belong to any error_group. The transactions succeeded, they have NA/NULL in here is correct.
- campaign_code:
- campaign_type
- model:
- platform:


In [ ]:
df_join_all[df_join_all['error_group'].notnull()]

,status_id,status_name,description,error_group
0,1,SUCCESS,Transaction successful,NaN
1,-1,INVALID_TIME,Payment overdue,customer
2,-2,LOW_BALANCE,Insufficient funds in customer account. Please...,customer
3,-3,INVALID_BANK,An error occurred. Please try again later.,external
4,-4,WRONG_PIN_EXCEEDED,Password locked due to multiple incorrect atte...,customer
5,-5,FAILED_CHARGE,Payment failed,external
6,-6,KYC_REQUIRED_USER,Please KYC to proceed with transactions,internal
7,-7,TRANSACTION_LIMITED,Transaction temporarily limited,internal
